In [1]:
# randomgrowth.jl
using Plots
using Interact

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-3959666757950267918\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-4562046518762753355\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-3959666757950267918\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [69]:
"Find squares that can possibly grow at next step in matrix G"
function possible_squares(G)
    M, N = size(G)
    squares = Tuple{Int,Int}[]  
    
    for ii = 1:M
        for jj = 1:N
            if G[ii, jj] == 0
                if  (ii == 1 && jj > 1 && G[ii, jj-1] == 1) ||
                    (jj == 1 && ii > 1 && G[ii-1, jj] == 1) ||
                    (ii > 1 && jj > 1 && G[ii-1, jj] == 1 && G[ii, jj-1] == 1)
                    push!(squares, (ii, jj))
                end
            end
        end
    end
    
    return squares
end

possible_squares

In [109]:
"""
Calculate the dynamics of random growth.

Returns a Vector of heatmaps containing the system dynamics over time.
"""

const possible = 0.4
const successful = 0.6

function calculate_random_growth(M, N, q, T=5000)

    G = zeros(M, N)

    all_heatmaps = typeof(G)[]

    # initial condition: bottom left square on
    G[1, 1] = true  
    push!(all_heatmaps, copy(G))

    for t = 1:T
        squares = possible_squares(G)
        
        # highlight the possible squares:
        for square in squares
            G[square[1], square[2]] = possible
        end

        push!(all_heatmaps, copy(G))


        # choose squares to actually grow and highlight them:
        for square in squares
            if rand() > q   # grows with probability p = 1 - q
                G[square[1], square[2]] = successful
            end
        end

        push!(all_heatmaps, copy(G))
        
        # finish growth:
        for i in 1:M
            for j in 1:N
                if G[i, j] == successful
                    G[i, j] = 1    # complete growth
                    
                elseif G[i, j] == possible
                    G[i, j] = 0    # square dies if growth doesn't happen
                end
            end
        end
        
        push!(all_heatmaps, copy(G))

        # vectorized version:
#         G[G .== 0.5] .= 1
#         G[G .== 0.25] .= 0

    end

    return all_heatmaps
end

calculate_random_growth (generic function with 2 methods)

In [125]:
# analytical limiting shape:
limiting(x, y) = x + y + 2*√(x*y*q)

limiting (generic function with 1 method)

In [135]:
function visualize_random_growth(M, N, q, T=100)
    
    all_heatmaps = calculate_random_growth(M, N, q, T)

    @manipulate for i in slider( 1:length(all_heatmaps), value=1)
        
        t = (i - 1) ÷ 3 
        
        # 3 slider movements for each time step
        
        p = heatmap(all_heatmaps[i], aspect_ratio=1, colorbar=false, clims=(0, 1), 
                    xlims = (0, N), ylims=(0, M))
        
        # p = contour(all_heatmaps[i], levels=[1])
        
        plot!(title="t = $t")
        
        range = t .* (0:0.001:(1-q))
        
        if t > 0   # draw limiting shape
            contour!(range, range, 
                     (x, y) -> limiting(x/t, y/t), levels=[1-q], 
                     colorbar=false, color=:blue, lw=3)
        end
        
        
        p
        
    end
    
end

visualize_random_growth (generic function with 2 methods)

In [136]:
q = 0.5
visualize_random_growth(20, 20, q, 1000)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[nothing], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 3001,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Int64} with 2 listeners. Value:
1, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/dpsanders/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/dpsanders/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/dpsanders/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/dpsanders/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/dpsanders/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000001332d5f90, Task (runnable) @0x00000001332d5f90), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\

In [137]:
function visualize_random_growth_scaled(M, N, q, T=100)
    
    all_heatmaps = calculate_random_growth(M, N, q, T)

    @manipulate for i in slider( 1:length(all_heatmaps), value=1)
        
        t = (i - 1) ÷ 3 + 3   # 3 slider movements for each time step
        
        p = heatmap(0:(1/(t-1)):1, 0:(1/(t-1)):1, all_heatmaps[i][1:t, 1:t], aspect_ratio=1, colorbar=false, clims=(0, 1), 
                    xlims = (0, 1), ylims=(0, 1))
        
        # p = contour(all_heatmaps[i], levels=[1])
        
        plot!(title="t = $t")
                

        range = 0:0.001:1
        contour!(range, range,
                 (x, y) -> limiting(x, y), levels=[1-q], colorbar=false, 
                 color=:blue, lw=3)
        
        
        p
    end
        
end

visualize_random_growth_scaled (generic function with 2 methods)

In [138]:
q = 0.5
visualize_random_growth_scaled(200, 200, q, 1000)

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any[nothing], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 3001,:min => 1,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Int64} with 2 listeners. Value:
1, nothing)), Set(String[]), nothing, Asset[Asset("js", "knockout", "/Users/dpsanders/.julia/packages/Knockout/1sDlc/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/Users/dpsanders/.julia/packages/Knockout/1sDlc/src/../assets/knockout_punches.js"), Asset("js", nothing, "/Users/dpsanders/.julia/packages/InteractBase/9mFwe/src/../assets/all.js"), Asset("css", nothing, "/Users/dpsanders/.julia/packages/InteractBase/9mFwe/src/../assets/style.css"), Asset("css", nothing, "/Users/dpsanders/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set(AbstractConnection[]), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00000001332d6ad0, Task (runnable) @0x00000001332d6ad0), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"1\",\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\",\"11\",\"12\",\"13\",\"14\",\"15\",\"16\",\"17\",\"18\",\"19\",\"20\",\"21\",\"22\",\"23\",\"24\",\"25\",\"26\",\"27\",\"28\",\"29\",\"30\",\"31\",\"32\",\"33\",\

BoundsError: BoundsError: attempt to access 200×200 Array{Float64,2} at index [1:211, 1:211]

BoundsError: attempt to access 200×200 Array{Float64,2} at index [1:211, 1:211]
Stacktrace:
 [1] throw_boundserror(::Array{Float64,2}, ::Tuple{UnitRange{Int64},UnitRange{Int64}}) at ./abstractarray.jl:538
 [2] checkbounds at ./abstractarray.jl:503 [inlined]
 [3] _getindex at ./multidimensional.jl:669 [inlined]
 [4] getindex at ./abstractarray.jl:981 [inlined]
 [5] (::getfield(Main, Symbol("##197#199")){Float64,Array{Array{Float64,2},1}})(::Int64) at ./In[137]:9
 [6] (::getfield(Observables, Symbol("##16#17")){getfield(Main, Symbol("##197#199")){Float64,Array{Array{Float64,2},1}},Observable{Any}})(::Int64) at /Users/dpsanders/.julia/packages/Observables/qCJWB/src/Observables.jl:152
 [7] (::getfield(Observables, Symbol("#g#15")){getfield(Observables, Symbol("##16#17")){getfield(Main, Symbol("##197#199")){Float64,Array{Array{Float64,2},1}},Observable{Any}},Tuple{Widget{:slider,Int64}}})(::Int64) at /Users/dpsanders/.julia/packages/Observables/qCJWB/src/Observables.jl:136
 [8] #setindex!#1